### 上一小节，我们通过示例，主要讲解了如何使用pandas对单表进行数据分析，统计等操作.

### 本节，我们将通过另一个示例，讲解多表连接后，对数据进行清洗，分析等操作.

### 示例数据采用美国各州人口统计数据，一共有三张统计数据表分别为：
- state-abbrevs.csv 
- state-population.csv 
- state-areas.csv

- 本次分享的主要任务包括以下几个方面：
    - 导入文件，查看原始数据
    - 将人口数据和各州简称数据进行合并
    - 将合并的数据中重复的abbreviation列进行删除
    - 查看存在缺失数据的列
    - 找到有哪些state/region使得state的值为NaN，进行去重操作
    - 为找到的这些state/region的state项补上正确的值，从而去除掉state这一列的所有NaN
    - 合并各州面积数据areas
    - 我们会发现area(sq.mi)这一列有缺失数据，找出是哪些行
    - 去除含有缺失数据的行
    - 找出2010年的全民人口数据
    - 计算各州的人口密度
    - 排序，并找出人口密度最高的五个州

In [1]:
# 导入必要的包
import numpy as np
import pandas as pd

#### 导入文件，查看原始数据

In [2]:
# 分别读取三个csv文件 state-abbrevs.csv state-population.csv state-areas.csv
abb = pd.read_csv('state-abbrevs.csv')
pop = pd.read_csv('state-population.csv')
area = pd.read_csv('state-areas.csv')

In [3]:
abb.head(3)

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ


In [4]:
pop.head(3)

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0


In [5]:
area.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


#### 将人口数据和各州简称数据进行合并

In [6]:
abb_pop = pd.merge(pop,abb,left_on='state/region',right_on='abbreviation',how='outer')
abb_pop.head(3)

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL


In [7]:
abb_pop.head(10)

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL
3,AL,total,2010,4785570.0,Alabama,AL
4,AL,under18,2011,1125763.0,Alabama,AL
5,AL,total,2011,4801627.0,Alabama,AL
6,AL,total,2009,4757938.0,Alabama,AL
7,AL,under18,2009,1134192.0,Alabama,AL
8,AL,under18,2013,1111481.0,Alabama,AL
9,AL,total,2013,4833722.0,Alabama,AL


#### 将合并的数据中重复的abbreviation列进行删除

In [8]:
abb_pop.drop(labels='abbreviation',axis=1,inplace=True)
abb_pop.head(3)

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama


#### 查看存在缺失数据的列

In [9]:
abb_pop.isnull().head(3)
abb_pop.notnull().head(3)

,state/region,ages,year,population,state
0,True,True,True,True,True
1,True,True,True,True,True
2,True,True,True,True,True


In [10]:
abb_pop.isnull().any(axis=0)

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [11]:
abb_pop.notnull().all(axis=0)

state/region     True
ages             True
year             True
population      False
state           False
dtype: bool

#### 找到有哪些state/region使得state的值为NaN，进行去重操作(哪些简称对应的全程值为空，且对符合要求的简称进行去重)

In [12]:
# 找出state的空对应的简称数据
abb_pop.loc[abb_pop['state'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


In [13]:
# 对符合要求的简称进行去重
abb_pop.loc[abb_pop['state'].isnull()]['state/region'].unique()

array(['PR', 'USA'], dtype=object)

#### 为找到的这些state/region的state项补上正确的值，从而去除掉state这一列的所有NaN
    - PR和USA对应的全称的值为NAN

In [14]:
# 找出符合要求行数据的行索引 PR == Puerto Rico | USA == united states
index = abb_pop.loc[abb_pop['state/region']=='PR'].index
# 将indexs对应的行中的state列的空值批量赋值成Puerto Rico
abb_pop.loc[index,'state'] = 'Puerto Rico'

In [15]:
# 对简称为USA的行采用同样的操作，去除空值
index = abb_pop.loc[abb_pop['state/region']=='USA'].index
abb_pop.loc[index,'state'] = 'united states'

In [16]:
abb_pop.loc[abb_pop['state'].isnull()]['state/region'].unique()

array([], dtype=object)

In [17]:
abb_pop.loc[abb_pop['state/region']=='USA'].head()
abb_pop.loc[abb_pop['state/region']=='PR'].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,Puerto Rico
2449,PR,total,1990,NaN,Puerto Rico
2450,PR,total,1991,NaN,Puerto Rico
2451,PR,under18,1991,NaN,Puerto Rico
2452,PR,total,1993,NaN,Puerto Rico


#### 合并各州面积数据areas

In [18]:
# show abb_pop
abb_pop.head(3)

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama


In [19]:
# show area
area.head(3)

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006


In [20]:
abb_pop_area = pd.merge(abb_pop,area,how='outer')
abb_pop_area.head(3)

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0


#### 我们会发现area(sq.mi)这一列有缺失数据，找出是哪些行

In [21]:
abb_pop_area.isnull().any(axis=0)

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [22]:
index = abb_pop_area.loc[abb_pop_area['area (sq. mi)'].isnull()].index
index

Int64Index([2496, 2497, 2498, 2499, 2500, 2501, 2502, 2503, 2504, 2505, 2506,
            2507, 2508, 2509, 2510, 2511, 2512, 2513, 2514, 2515, 2516, 2517,
            2518, 2519, 2520, 2521, 2522, 2523, 2524, 2525, 2526, 2527, 2528,
            2529, 2530, 2531, 2532, 2533, 2534, 2535, 2536, 2537, 2538, 2539,
            2540, 2541, 2542, 2543],
           dtype='int64')

#### 去除含有缺失数据的行

In [23]:
abb_pop_area.drop(labels=index,axis=0,inplace=True)

In [24]:
abb_pop_area.loc[abb_pop_area['area (sq. mi)'].isnull()].index

Int64Index([], dtype='int64')

#### 找出2010年的全民人口数据

In [25]:
abb_pop_area.index = abb_pop_area['year']
abb_pop_area.head(3)

,state/region,ages,year,population,state,area (sq. mi)
year,,,,,,
2012,AL,under18,2012,1117489.0,Alabama,52423.0
2012,AL,total,2012,4817528.0,Alabama,52423.0
2010,AL,under18,2010,1130966.0,Alabama,52423.0


In [26]:
# 找到2010年的数据，再找total的数据
abb_pop_area_2010 = abb_pop_area.loc[2010]
abb_pop_area_2010.loc[abb_pop_area_2010['ages']=='total']['population'].sum()

313047503.0

In [27]:
# 使用牛逼的query
abb_pop_area.query('ages=="total" & year=="2010"')['population'].sum()

313047503.0

#### 赠送内容：把索引变为日期时间类型

In [28]:
# abb_pop_area.index.year
# type(abb_pop_area.index.year)

#### 计算各州的人口密度

In [29]:
abb_pop_area['pop_des'] = abb_pop_area['population'] / abb_pop_area['area (sq. mi)']
abb_pop_area.head(3)

,state/region,ages,year,population,state,area (sq. mi),pop_des
year,,,,,,,
2012,AL,under18,2012,1117489.0,Alabama,52423.0,21.316769
2012,AL,total,2012,4817528.0,Alabama,52423.0,91.897221
2010,AL,under18,2010,1130966.0,Alabama,52423.0,21.573851


#### 排序，并找出2013年人口密度最高的五个州

In [30]:
# 获取2013年各州人口总数数据信息
abb_pop_area_2013 = abb_pop_area.query('ages=="total" & year=="2013"')
abb_pop_area_2013.head()

,state/region,ages,year,population,state,area (sq. mi),pop_des
year,,,,,,,
2013,AL,total,2013,4833722.0,Alabama,52423.0,92.206131
2013,AK,total,2013,735132.0,Alaska,656425.0,1.119903
2013,AZ,total,2013,6626624.0,Arizona,114006.0,58.125221
2013,AR,total,2013,2959373.0,Arkansas,53182.0,55.646140
2013,CA,total,2013,38332521.0,California,163707.0,234.153219


In [31]:
# 2013年人口密度最高的五个州
top_five = abb_pop_area_2013.sort_values(by='pop_des',ascending=False)[['state','pop_des']].head()
top_five

,state,pop_des
year,,
2013,District of Columbia,9506.602941
2013,Puerto Rico,1028.473969
2013,New Jersey,1020.332378
2013,Rhode Island,680.589644
2013,Connecticut,648.643579


In [32]:
# set state as index
top_five = top_five.set_index('state')

In [33]:
# show result as a dictionary
top_five

,pop_des
state,
District of Columbia,9506.602941
Puerto Rico,1028.473969
New Jersey,1020.332378
Rhode Island,680.589644
Connecticut,648.643579


In [34]:
top_dict = top_five.to_dict()
top_dict

{'pop_des': {'District of Columbia': 9506.60294117647,
  'Puerto Rico': 1028.4739687055476,
  'New Jersey': 1020.3323778949782,
  'Rhode Island': 680.589644012945,
  'Connecticut': 648.6435786435786}}

In [35]:
top_dict['pop_des'].keys()

dict_keys(['District of Columbia', 'Puerto Rico', 'New Jersey', 'Rhode Island', 'Connecticut'])